In [39]:
from fastembed import TextEmbedding

In [40]:
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [3]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
embed_model = TextEmbedding(model_name=model_handle)

In [33]:
plain_text = ['cat','dog','machine','computer']

In [34]:
text_embeddings = []
for text in plain_text:
    array_embd = embed_model.embed(text)
    embeddings = list(array_embd)
    text_embeddings.extend(embeddings)

In [38]:
query = "animal"
query_embeddings = list(embed_model.embed(query))
for i,vector in enumerate(text_embeddings):
    print(query + "->" + plain_text[i] + " : " + str(query_embeddings[0].dot(vector)))

animal->cat : 0.912273287263012
animal->dog : 0.963033842112502
animal->machine : 0.8520619227172069
animal->computer : 0.8483616946996702


In [36]:
cosine_similarity

[0.912273287263012, 0.963033842112502, 0.8520619227172069, 0.8483616946996702]

In [56]:
from fastembed import TextEmbedding
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [71]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [72]:
collection_name = "zoomcamp-course-faq"
client.delete_collection(collection_name) #delete if exists
# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [73]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()
documents = []

for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [75]:
points = []
for i, doc in enumerate(documents):
    text = doc['text'] 
    # You can make the vector from both(question and answer) doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [76]:
len(points)

948

In [77]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [78]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [83]:
import random
import json
course_piece = random.choice(documents)
print(json.dumps(course_piece, indent=2))

{
  "text": "If below does not work, then download the latest available py4j version with\nconda install -c conda-forge py4j\nTake care of the latest version number in the website to replace appropriately.\nNow add\nexport PYTHONPATH=\"${SPARK_HOME}/python/:$PYTHONPATH\"\nexport PYTHONPATH=\"${SPARK_HOME}/python/lib/py4j-0.10.9.7-src.zip:$PYTHONPATH\"\nin your  .bashrc file.",
  "section": "Module 5: pyspark",
  "question": "Py4J Error - ModuleNotFoundError: No module named 'py4j' (Solve with latest version)",
  "course": "data-engineering-zoomcamp"
}


In [87]:
result = search(course_piece['question'])

In [88]:
result

QueryResponse(points=[ScoredPoint(id=918, version=0, score=0.8446106, payload={'text': 'i\'m getting this error ModuleNotFoundError: No module named \'pipenv.patched.pip._vendor.urllib3.response\'\nand Resolved from this command pip install pipenv --force-reinstall\ngetting this errror site-packages\\pipenv\\patched\\pip\\_vendor\\urllib3\\connectionpool.py"\nResolved from this command pip install -U pip and pip install requests\nAsif', 'section': 'Module 5: Monitoring', 'question': "ModuleNotFoundError: No module named 'pipenv.patched.pip._vendor.urllib3.response'", 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [89]:
print(f"Question:\n{course_piece['question']}\n")
print("Top Retrieved Answer:\n{}\n".format(result.points[0].payload['text']))
print("Original Answer:\n{}".format(course_piece['text']))

Question:
Py4J Error - ModuleNotFoundError: No module named 'py4j' (Solve with latest version)

Top Retrieved Answer:
i'm getting this error ModuleNotFoundError: No module named 'pipenv.patched.pip._vendor.urllib3.response'
and Resolved from this command pip install pipenv --force-reinstall
getting this errror site-packages\pipenv\patched\pip\_vendor\urllib3\connectionpool.py"
Resolved from this command pip install -U pip and pip install requests
Asif

Original Answer:
If below does not work, then download the latest available py4j version with
conda install -c conda-forge py4j
Take care of the latest version number in the website to replace appropriately.
Now add
export PYTHONPATH="${SPARK_HOME}/python/:$PYTHONPATH"
export PYTHONPATH="${SPARK_HOME}/python/lib/py4j-0.10.9.7-src.zip:$PYTHONPATH"
in your  .bashrc file.


In [90]:
print(search("What if I submit homeworks late?").points[0].payload['text'])


No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]


In [91]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [92]:
def search_in_course(query, course="mlops-zoomcamp", limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter( # filter by course name
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [93]:
print(search_in_course("What if I submit homeworks late?", "mlops-zoomcamp").points[0].payload['text'])


Please choose the closest one to your answer. Also do not post your answer in the course slack channel.


In [94]:
print(search_in_course("What if I submit homeworks late?", "data-engineering-zoomcamp").points[0].payload['text'])


No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]


In [104]:
from fastembed import TextEmbedding
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
words = ["cat", "dog", "machine", "computer"]
embeddings = [model.embed(word) for word in words]
query = list(model.embed("animal"))
for i, word in enumerate(words):
    sim = query[0].dot(list(embeddings[i])[0])
    print(f"animal -> {word}: {sim:.3f}")

animal -> cat: 0.912
animal -> dog: 0.963
animal -> machine: 0.852
animal -> computer: 0.848
